In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow

In [0]:
# KaggleのTGS Salt Identification Challengeのデータセットを使用し、セグメンテーション

In [1]:
# 自分のマイドライブにマウントする

from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
pwd

'/content'

In [2]:
cd drive/My\ Drive

/content/drive/My Drive


In [0]:
pwd

'/content/drive/My Drive'

In [0]:
# mkdir .kaggle

In [0]:
# ls -a

./  ../  .config/  drive/  .kaggle/  sample_data/


In [0]:
# import json

# token ={"username":"kyuuy1001","key":"48a86cfd1d773db8df0f37362ec22a4e"}
# with open('/content/.kaggle/kaggle.json', 'w') as file:
#     json.dump(token, file)

In [0]:
#!chmod 600 /content/.kaggle/kaggle.json

In [0]:
#mkdir　/root/.kaggle/

In [0]:
#!cp -R /content/.kaggle/kaggle.json /root/.kaggle/

In [0]:
#!ls -a /root/.kaggle/

.  ..  kaggle.json


In [0]:
# kaggle APIコマンドが使えるかどうかを確認
#!kaggle competitions list

ref                                            deadline             category             reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ----------  ---------  --------------  
digit-recognizer                               2030-01-01 00:00:00  Getting Started   Knowledge       2188           False  
titanic                                        2030-01-01 00:00:00  Getting Started   Knowledge      16538            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started   Knowledge       4507            True  
connectx                                       2030-01-01 00:00:00  Getting Started   Knowledge        328           False  
competitive-data-science-predict-future-sales  2020-12-31 23:59:00  Playground            Kudos       6024           False  
m5-forecasting-accuracy                        2020-06-30 23:59:00  Featured            $50,000       1205            True  


In [0]:
# # カレントディレクトリの変更

# import os

# os.chdir('/content/drive/My Drive/')

# # カレントディレクトリの取得
# print(os.getcwd())

/content/drive/My Drive


In [0]:
# !kaggle competitions download -c tgs-salt-identification-challenge

In [0]:
#!unzip /content/drive/'My Drive'/competition_data.zip  -d Salt

In [0]:
#!unzip /content/drive/'My Drive'/flamingo.zip  -d Salt

# 【問題1】学習・推定
以下のKeras実装を使用して学習・推定を行ってください。



In [0]:
#!git clone https://github.com/zhixuhao/unet.git

Cloning into 'unet'...
remote: Enumerating objects: 394, done.
remote: Total 394 (delta 0), reused 0 (delta 0), pack-reused 394
Receiving objects: 100% (394/394), 44.92 MiB | 2.69 MiB/s, done.
Resolving deltas: 100% (79/79), done.
Checking out files: 100% (253/253), done.


In [0]:
cd unet

/content/drive/My Drive/unet


In [0]:
!python main.py

Using TensorFlow backend.






2020-03-23 10:07:04.736274: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-03-23 10:07:04.736458: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2224a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-03-23 10:07:04.736493: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-03-23 10:07:04.738379: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-23 10:07:04.889529: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-03-23 10:07:04.890292: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2224bc0 initialized for platform CUDA (this does not guarantee that XLA will be use

### 変更備忘録
1.model.py :畳み込みごに「conv1 = BatchNormalization()(conv1)」を追加することで、予測データが真っ黒になることを回避できるとあったので変更<br>
2.main.py :各読み込みフォルダのパス変更<br>
3.data.py :def tesstGeneratorに「％d」部分が整数値のみの指定なので今回のデータにはふさわしくない。そのためlistdirでデータ名取得<br> 

# 【問題2】コードリーディング
論文[1]に目を通した上で、上記実装のコードリーディングを行ってください。

### 論文上、重要そうな箇所<br>
>１、全結合層利用しない。全て畳み込みで行なっている＊P3<br>
>２、画像の境界領域のピクセルを予測するために、入力画像をミラーリングすることにより、欠落しているコンテキストを推定します。＊P3<br>
>３、ダウンサンプリングの１つは、２つの３＊３Conv、Relu、２＊２Maxpoolingからなっている。<br>
＊It consists of the repeated application of two 3x3 convolutions (unpadded convolutions), each followed by a rectified linear unit (ReLU) and a 2x2 max pooling operation with stride 2 for downsampling.

In [0]:
# U-net構造のコード
def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    # skip-conection
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    # skip-conection
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    # skip-conection
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    # skip-conection
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model


gitのissueで話題になっていたのが、予測画像がグレースケールで出てくる。
解決策に上がっていたのが、閾値を変えること

In [0]:
   for i,item in enumerate(npyfile):
      if flag_multi_class:`
            img = labelVisualize(num_class,COLOR_DICT,item)
      else:
            img=item[:,:,0]
            print(np.max(img),np.min(img))
            img[img>0.5]=1
            img[img<=0.5]=0
            print(np.max(img),np.min(img))
            io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

その他、data.pyでは画像変換など行われているので、この部分を自分の入力する画像に合わせ編集する必要がある。